# Exercices : Communication Anonyme

**Objectifs** :
- Analyser la sécurité de Tor
- Attaquer des mixnets mal conçus
- Comprendre les limites de l'anonymat
- Implémenter des protections

In [ ]:
import secrets
import hashlib
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

## Exercice 1 : Traffic Analysis - Corrélation Temporelle

**Contexte** : Un adversaire observe le trafic à l'entrée et à la sortie du réseau Tor.

**Tâche** : Simulez une attaque par corrélation de timing.

In [ ]:
def traffic_correlation_attack():
    """
    Simulation d'attaque par corrélation de trafic.
    """
    print("=" * 70)
    print("ATTAQUE : Corrélation de Trafic (Timing)")
    print("=" * 70)
    
    # Simulation : 10 utilisateurs Tor
    n_users = 10
    n_connections = 100
    
    print(f"\n🌐 Simulation :")
    print(f"   Utilisateurs : {n_users}")
    print(f"   Connexions : {n_connections}")
    
    # Générer trafic : timestamp d'entrée et de sortie
    # Chaque utilisateur a un pattern unique de connexions
    
    users_entry_times = {}
    users_exit_times = {}
    
    for user_id in range(n_users):
        # Pattern de connexion (times en secondes)
        base_time = user_id * 10
        entry_times = [base_time + i + np.random.normal(0, 0.5) for i in range(n_connections // n_users)]
        
        # Sortie : même pattern avec latence Tor (~200ms) + bruit
        latency = 0.2 + np.random.normal(0, 0.05)
        exit_times = [t + latency + np.random.normal(0, 0.05) for t in entry_times]
        
        users_entry_times[user_id] = sorted(entry_times)
        users_exit_times[user_id] = sorted(exit_times)
    
    # Adversaire observe tous les timings (Entry et Exit)
    all_entry_times = []
    all_exit_times = []
    
    for user_id in range(n_users):
        all_entry_times.extend([(t, user_id) for t in users_entry_times[user_id]])
        all_exit_times.extend([(t, user_id) for t in users_exit_times[user_id]])
    
    # Mélanger (adversaire ne connaît pas les user_id, seulement les timings)
    import random
    all_entry_times_anon = [t for t, _ in all_entry_times]
    all_exit_times_anon = [t for t, _ in all_exit_times]
    
    random.shuffle(all_entry_times_anon)
    random.shuffle(all_exit_times_anon)
    
    # Attaque : Corrélation par timing
    print(f"\n🔍 Adversaire tente de corréler entrées et sorties...")
    
    # Pour un utilisateur cible (user 0)
    target_user = 0
    target_entry_times = users_entry_times[target_user]
    target_exit_times = users_exit_times[target_user]
    
    print(f"\n🎯 Cible : Utilisateur {target_user}")
    print(f"   Connexions entrée : {len(target_entry_times)}")
    print(f"   Timing moyen : {np.mean(target_entry_times):.2f}s")
    
    # Calculer corrélation avec toutes les sorties possibles
    best_correlation = -1
    best_match = None
    
    for candidate_user in range(n_users):
        candidate_exit_times = users_exit_times[candidate_user]
        
        # Corrélation simple : comparer différences de timing
        # Si pattern similaire, corrélation élevée
        correlation = 0
        for i in range(len(target_entry_times) - 1):
            delta_entry = target_entry_times[i+1] - target_entry_times[i]
            
            for j in range(len(candidate_exit_times) - 1):
                delta_exit = candidate_exit_times[j+1] - candidate_exit_times[j]
                
                # Si deltas similaires, augmenter score
                if abs(delta_entry - delta_exit) < 0.5:
                    correlation += 1
        
        if correlation > best_correlation:
            best_correlation = correlation
            best_match = candidate_user
    
    print(f"\n📊 Résultat de l'attaque :")
    print(f"   Meilleure correspondance : Utilisateur {best_match}")
    print(f"   Score de corrélation : {best_correlation}")
    print(f"   Cible réelle : Utilisateur {target_user}")
    
    if best_match == target_user:
        print(f"\n✅ SUCCÈS : Utilisateur dé-anonymisé !")
    else:
        print(f"\n❌ ÉCHEC : Mauvaise identification")
    
    print(f"\n⚠️  LEÇON :")
    print(f"   - Corrélation de timing possible si adversaire contrôle Entry ET Exit")
    print(f"   - Probabilité augmente avec durée d'observation")
    print(f"   - Tor ajoute bruit/délais, mais pas toujours suffisant")
    
    print(f"\n✅ PROTECTIONS :")
    print(f"   - Cover traffic (trafic factice constant)")
    print(f"   - Padding pour uniformiser tailles de paquets")
    print(f"   - Délais aléatoires (augmente latence)")
    print(f"   - Réseau large (diminue probabilité contrôle Entry+Exit)")

traffic_correlation_attack()

## Exercice 2 : Website Fingerprinting

**Contexte** : Un adversaire observe le trafic chiffré et veut deviner quel site est visité.

**Tâche** : Simulez une attaque par fingerprinting basée sur la taille des paquets.

In [ ]:
def website_fingerprinting_attack():
    """
    Simulation d'attaque par fingerprinting de sites web.
    """
    print("\n" + "=" * 70)
    print("ATTAQUE : Website Fingerprinting")
    print("=" * 70)
    
    # Simuler signatures de sites (taille paquets)
    # Chaque site a un pattern caractéristique
    
    websites = {
        'news.com': [512, 512, 1024, 512, 2048, 512, 1024],  # Beaucoup d'images
        'search.com': [256, 256, 512, 256, 256],  # Page simple
        'video.com': [512, 1024, 4096, 4096, 4096, 4096],  # Streaming
        'social.com': [512, 512, 256, 256, 512, 256, 512],  # Feed dynamique
    }
    
    print(f"\n🌐 Base de données de signatures :")
    for site, signature in websites.items():
        avg_size = np.mean(signature)
        n_packets = len(signature)
        print(f"   {site:15} : {n_packets} paquets, taille moyenne = {avg_size:.0f} bytes")
    
    # Utilisateur visite un site via Tor
    target_site = 'video.com'
    target_signature = websites[target_site].copy()
    
    # Ajouter bruit (Tor padding, fragmentation)
    noisy_signature = [s + np.random.randint(-100, 100) for s in target_signature]
    
    print(f"\n🎯 Utilisateur visite : {target_site}")
    print(f"   Signature observée (bruitée) : {noisy_signature}")
    
    # Adversaire tente d'identifier le site
    print(f"\n🔍 Adversaire analyse le trafic chiffré...")
    
    # Méthode simple : Calculer distance avec chaque signature
    best_match = None
    best_score = float('inf')
    
    for site, signature in websites.items():
        # Distance : différence de taille totale + nombre de paquets
        if len(signature) != len(noisy_signature):
            # Pénalité si nombre de paquets différent
            distance = 10000
        else:
            # Distance euclidienne
            distance = np.sqrt(sum((s - n)**2 for s, n in zip(signature, noisy_signature)))
        
        print(f"   Distance à {site:15} : {distance:.2f}")
        
        if distance < best_score:
            best_score = distance
            best_match = site
    
    print(f"\n📊 Résultat :")
    print(f"   Prédiction : {best_match}")
    print(f"   Site réel : {target_site}")
    print(f"   Score : {best_score:.2f}")
    
    if best_match == target_site:
        print(f"\n✅ SUCCÈS : Site identifié !")
    else:
        print(f"\n❌ ÉCHEC : Mauvaise identification")
    
    print(f"\n⚠️  LEÇON :")
    print(f"   - Même avec chiffrement, patterns de trafic révélateurs")
    print(f"   - Attaques ML peuvent atteindre ~90% précision")
    print(f"   - Tor Browser fait du padding, mais pas parfait")
    
    print(f"\n✅ PROTECTIONS :")
    print(f"   - Padding constant (uniformiser tailles)")
    print(f"   - Dummy packets (compléter à taille fixe)")
    print(f"   - Traffic shaping (retarder/accélérer)")
    print(f"   - WTF-PAD (défense dans Tor Browser)")

website_fingerprinting_attack()

## Exercice 3 : (n-1) Attack sur Mixnet

**Contexte** : Un adversaire veut isoler un message cible dans un mix.

**Tâche** : Implémentez l'attaque (n-1) et démontrez son efficacité.

In [ ]:
def n_minus_1_attack():
    """
    Attaque (n-1) sur mixnet.
    """
    print("\n" + "=" * 70)
    print("ATTAQUE : (n-1) Attack sur Mixnet")
    print("=" * 70)
    
    batch_size = 10
    
    print(f"\n🔐 Mix Server :")
    print(f"   Batch size : {batch_size}")
    print(f"   Attend {batch_size} messages avant de mixer")
    
    # Utilisateur cible envoie un message
    print(f"\n👤 Alice envoie un message au mix")
    alice_message = "Secret message from Alice"
    
    # Adversaire (Eve) envoie (n-1) messages
    print(f"\n💀 Eve (adversaire) envoie {batch_size - 1} messages")
    print(f"   Tous avec destination connue d'Eve")
    
    eve_messages = [f"Eve dummy message {i}" for i in range(batch_size - 1)]
    eve_destinations = [f"eve_address_{i}" for i in range(batch_size - 1)]
    
    # Mix reçoit n messages
    print(f"\n📬 Mix reçoit {batch_size} messages :")
    print(f"   - 1 message d'Alice (cible)")
    print(f"   - {batch_size - 1} messages d'Eve")
    
    # Mix traite le batch
    print(f"\n🔄 Mix traite le batch :")
    print(f"   1. Déchiffre tous les messages")
    print(f"   2. Mélange aléatoirement")
    print(f"   3. Transmet dans ordre aléatoire")
    
    # Messages sortants
    all_destinations = ['alice_real_destination'] + eve_destinations
    
    import random
    random.shuffle(all_destinations)
    
    print(f"\n📤 Messages sortants (ordre aléatoire) :")
    for i, dest in enumerate(all_destinations, 1):
        print(f"   Message {i} → {dest}")
    
    # Eve observe les sorties
    print(f"\n🔍 Eve observe les sorties :")
    print(f"   Eve reconnaît ses {batch_size - 1} messages (destinations connues)")
    
    # Identifier message d'Alice
    alice_destination = None
    for dest in all_destinations:
        if dest not in eve_destinations:
            alice_destination = dest
            break
    
    print(f"   Le message restant doit être celui d'Alice !")
    print(f"\n✅ SUCCÈS : Eve a identifié la destination d'Alice")
    print(f"   Destination : {alice_destination}")
    
    print(f"\n💥 IMPACT :")
    print(f"   - Eve sait qu'Alice a envoyé un message à {alice_destination}")
    print(f"   - Anonymat d'Alice cassé !")
    
    print(f"\n⚠️  VULNÉRABILITÉ :")
    print(f"   - Adversaire peut contrôler (n-1) messages du batch")
    print(f"   - Isole le message cible par élimination")
    print(f"   - Fonctionne même si mix est honnête")
    
    print(f"\n✅ PROTECTIONS :")
    print(f"   1. Dummy traffic : Mix ajoute messages factices")
    print(f"      → Batch contient toujours messages inconnus d'Eve")
    print(f"   2. Batch size grand (>100)")
    print(f"      → Plus difficile pour Eve de contrôler (n-1) messages")
    print(f"   3. Délais aléatoires")
    print(f"      → Eve ne sait pas quand batch est traité")
    print(f"   4. Multiple batches")
    print(f"      → Message cible peut être dans batch différent")

n_minus_1_attack()

## Exercice 4 : Anonymat vs Performance

**Contexte** : Trade-off entre anonymat et performance.

**Tâche** : Analysez l'impact du nombre de relais/mixes sur l'anonymat et la latence.

In [ ]:
def anonymity_vs_performance():
    """
    Analyse du trade-off anonymat vs performance.
    """
    print("\n" + "=" * 70)
    print("ANALYSE : Anonymat vs Performance")
    print("=" * 70)
    
    # Simulation : Réseau de 1000 relais
    n_relays = 1000
    adversary_fraction = 0.10  # 10% contrôlés par adversaire
    n_adversary_relays = int(n_relays * adversary_fraction)
    
    print(f"\n🌐 Réseau :")
    print(f"   Relais totaux : {n_relays}")
    print(f"   Relais malveillants : {n_adversary_relays} ({adversary_fraction*100:.0f}%)")
    
    # Calculer probabilité de compromission selon longueur circuit
    circuit_lengths = [1, 2, 3, 4, 5, 6]
    
    results = []
    
    for length in circuit_lengths:
        # Probabilité que TOUS les relais du circuit soient malveillants
        p_all_malicious = adversary_fraction ** length
        
        # Probabilité qu'AU MOINS Entry ET Exit soient malveillants
        # (suffisant pour traffic analysis)
        p_entry_exit_malicious = adversary_fraction ** 2
        
        # Latence (estimation simplifiée)
        latency_per_hop = 50  # ms
        total_latency = length * latency_per_hop
        
        results.append({
            'length': length,
            'p_all': p_all_malicious,
            'p_entry_exit': p_entry_exit_malicious,
            'latency': total_latency
        })
    
    # Affichage
    print(f"\n📊 Impact de la longueur du circuit :")
    print(f"\n{'Longueur':>9} | {'P(tous mal.)':>13} | {'P(entry+exit)':>14} | {'Latence':>10}")
    print("-" * 60)
    
    for r in results:
        print(f"{r['length']:>9} | {r['p_all']:>12.2e} | {r['p_entry_exit']:>13.2e} | {r['latency']:>8} ms")
    
    # Visualisation
    plt.figure(figsize=(12, 5))
    
    # Subplot 1 : Probabilités
    plt.subplot(1, 2, 1)
    lengths = [r['length'] for r in results]
    p_all = [r['p_all'] for r in results]
    p_entry_exit = [r['p_entry_exit'] for r in results]
    
    plt.semilogy(lengths, p_all, 'o-', label='P(tous malveillants)', linewidth=2)
    plt.axhline(y=p_entry_exit[0], color='r', linestyle='--', label=f'P(entry+exit mal.) = {p_entry_exit[0]:.2e}')
    plt.xlabel('Longueur du circuit')
    plt.ylabel('Probabilité de compromission (log)')
    plt.title('Sécurité vs Longueur')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Subplot 2 : Latence
    plt.subplot(1, 2, 2)
    latencies = [r['latency'] for r in results]
    plt.plot(lengths, latencies, 's-', color='green', linewidth=2)
    plt.axhline(y=200, color='orange', linestyle='--', label='Acceptable (~200ms)')
    plt.xlabel('Longueur du circuit')
    plt.ylabel('Latence totale (ms)')
    plt.title('Performance vs Longueur')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n💡 OBSERVATIONS :")
    print(f"   - Plus le circuit est long, plus l'anonymat est fort")
    print(f"   - Mais latence augmente linéairement")
    print(f"   - Tor utilise 3 relais (compromis)")
    print(f"   - P(all malicious) avec 3 relais : {results[2]['p_all']:.2e}")
    print(f"   - P(entry+exit) constant : {p_entry_exit[0]:.2e}")
    
    print(f"\n🎯 Choix de Tor (3 relais) :")
    print(f"   ✅ Bon équilibre sécurité/performance")
    print(f"   ✅ Latence acceptable (~150-200ms)")
    print(f"   ✅ P(compromission totale) très faible")
    print(f"   ⚠️  Traffic analysis reste possible (entry+exit)")

anonymity_vs_performance()

## Exercice 5 : Coût de l'Anonymat

**Contexte** : Quantifier les ressources nécessaires pour anonymat réseau-scale.

**Tâche** : Calculez le coût (bande passante, latence) pour différents systèmes.

In [ ]:
def cost_of_anonymity():
    """
    Analyse du coût de l'anonymat.
    """
    print("\n" + "=" * 70)
    print("ANALYSE : Coût de l'Anonymat")
    print("=" * 70)
    
    # Paramètres
    n_users = 2_000_000  # Utilisateurs quotidiens Tor
    avg_bandwidth_per_user = 1  # Mbps
    
    print(f"\n📊 Paramètres :")
    print(f"   Utilisateurs : {n_users:,}")
    print(f"   Bande passante moyenne : {avg_bandwidth_per_user} Mbps/utilisateur")
    
    # Comparaison systèmes
    systems = {
        'Direct (pas anonymat)': {
            'overhead': 1.0,
            'latency': 20,  # ms
            'anonymity': 0
        },
        'VPN': {
            'overhead': 1.1,  # 10% overhead (chiffrement)
            'latency': 30,  # ms
            'anonymity': 1  # Faible (VPN connaît identité)
        },
        'Tor (3 relais)': {
            'overhead': 1.3,  # 30% overhead (3 couches)
            'latency': 150,  # ms
            'anonymity': 8  # Bonne
        },
        'Mixnet (5 mixes)': {
            'overhead': 1.5,  # 50% overhead (batching, padding)
            'latency': 60000,  # ms (1 minute pour batch)
            'anonymity': 10  # Excellente
        },
    }
    
    # Calculs
    print(f"\n📈 Coût par système :")
    print(f"\n{'Système':20} | {'Overhead':>10} | {'Latence':>12} | {'Anonymat':>10} | {'Bande passante':>17}")
    print("-" * 90)
    
    for name, params in systems.items():
        total_bandwidth = n_users * avg_bandwidth_per_user * params['overhead']
        print(f"{name:20} | {params['overhead']:>9.1f}x | {params['latency']:>10} ms | {params['anonymity']:>9}/10 | {total_bandwidth:>13,.0f} Mbps")
    
    # Coût monétaire (estimation)
    print(f"\n💰 Coût monétaire estimé (par jour) :")
    cost_per_gbps_per_month = 500  # USD
    hours_per_day = 24
    days_per_month = 30
    
    for name, params in systems.items():
        total_bandwidth_gbps = n_users * avg_bandwidth_per_user * params['overhead'] / 1000
        monthly_cost = total_bandwidth_gbps * cost_per_gbps_per_month
        daily_cost = monthly_cost / days_per_month
        
        print(f"   {name:20} : ${daily_cost:>10,.0f} / jour (${monthly_cost:>10,.0f} / mois)")
    
    print(f"\n🎯 CONCLUSIONS :")
    print(f"   - Anonymat a un coût en performance ET en argent")
    print(f"   - Trade-off à faire selon cas d'usage :")
    print(f"      • Navigation web : Tor (latence acceptable)")
    print(f"      • Email anonyme : Mixnet (latence acceptable)")
    print(f"      • Streaming vidéo : VPN ou direct (latence critique)")
    print(f"   - Tor = bon compromis pour usage général")
    print(f"   - Mixnet = meilleur anonymat mais trop lent pour temps réel")
    
    print(f"\n📚 RESSOURCES TOR (2025) :")
    print(f"   - Relais : ~7000-8000 bénévoles")
    print(f"   - Bande passante : ~400 Gbps total")
    print(f"   - Coût estimé : ~$6M / an (dons)")
    print(f"   - Utilisateurs : ~2M quotidiens")
    print(f"   - Coût par utilisateur : ~$3 / an")

cost_of_anonymity()

## Conclusion

**Points clés vérifiés** :
- ❌ Traffic analysis : Corrélation possible si adversaire contrôle Entry+Exit
- ❌ Website fingerprinting : Patterns révélateurs même avec chiffrement
- ❌ (n-1) attack : Isolation de message dans mixnet
- ✅ Trade-off anonymat/performance : Tor = 3 relais (compromis)
- ✅ Coût de l'anonymat : Significatif mais gérable

**Retenir** :
- Anonymat parfait n'existe pas (surtout contre adversaire global)
- Protection dépend du modèle de menace (adversaire local vs global)
- Tor : Excellent contre surveillance locale/ISP
- Mixnet : Meilleur anonymat mais latence élevée
- Bonnes pratiques essentielles (HTTPS, pas de plugins, etc.)

**Applications pratiques** :
- Dissidents/journalistes : Tor + bonnes pratiques
- Email anonyme : Mixnet (si encore actifs)
- Vote électronique : Mixnet + vérifiabilité
- Navigation privée : VPN (suffisant pour publicités/tracking)

**Recherche active** :
- Loopix : Mixnet moderne avec cover traffic
- Nym : Mixnet avec incitations économiques
- Vuvuzela : Private messaging à grande échelle
- Signal : End-to-end encryption (complément à Tor)